__[Open and try this file online (Colab)](https://colab.research.google.com/github/djekra/wiktionary_wortschatz/blob/master/1_wiktionary_parsen/x10_d00_wiktionary_de_parser.ipynb)__

# wiktionary_de_parser
* erstellt die Datei wiktionary_vvv.pickle in data_00
* wendet wiktionary_de_parser an
* braucht ca. 15 Minuten

### Doku
* page_id <=> title eineindeutig
* page_part ist die Abschnittsnummer der Seite
* section_id2 identifiziert page_id-page_part-Kombination (heißt später lemma_id)

## Import

In [1]:
# blab init
try:
    import blab
except ImportError as e:
    !pip install blab
    import blab    
startup_notebook = blab.blab_startup()
%run $startup_notebook 

blab init
environment['in_colab']     = False
environment['dropbox_path'] = /home/me/Data_Linux/Dropbox
environment['lib_path']     = /home/me/Data_Linux/Dropbox/31_Projekte/01_Python/libs
Start Time: 20:07:10


time: 633 ms


In [ ]:
# pandas
if environment['in_colab']:
    !pip install --upgrade pandas
import pandas as pd

In [2]:
# alle anderen
try:
    import pandasklar as pak, bj_nlp, wiktionary_de_parser, wget, bz2file

except ImportError as e:
    !pip install pandasklar bj_nlp wiktionary_de_parser wget bz2file
    import pandasklar as pak, bj_nlp, wiktionary_de_parser, wget, bz2file  
    
grid       = pak.grid
check_mask = pak.check_mask        


time: 5.76 s


## Einstellungen

In [3]:
# Was debuggen?
suche_debug = ['entlassen']

# Load
verzeichnis_load          = 'data_00'
bzfile_url                = 'https://dumps.wikimedia.org/dewiktionary/latest/'
bzfile_filenam            = 'dewiktionary-latest-pages-articles-multistream.xml.bz2'
bzfile_filename           = os.path.join( verzeichnis_load, bzfile_filenam ) 

# Was parsen?      
min_id                    = 0         # 81: Mensch
max_id                    = 999999999 #22600 
version                   = '_all'
#version                  = '_short'


# Save
verzeichnis_save          = 'data_00'
wiktionary_filename       = os.path.join( verzeichnis_save, 'wiktionary'       +version+'.pickle' )       
wiktionary_trash_filename = os.path.join( verzeichnis_save, 'wiktionary_trash' +version+'.pickle' )  

time: 39.2 ms


In [4]:
# verbose
pak.Config.set('VERBOSE', True)

VERBOSE = True
--> setting verbose=True as default for all pandasklar functions

time: 37 ms


## Vorbereiten

In [5]:
# Verzeichnisse erzeugen
os.makedirs(verzeichnis_load, exist_ok=True) 
os.makedirs(verzeichnis_save, exist_ok=True) 

time: 44.4 ms


In [6]:
# download bzfile if necessary
# 1 min

if not os.path.isfile(bzfile_filename):
    filename = wget.download(bzfile_url + bzfile_filenam, out=verzeichnis_load)
else:
    print(bzfile_filenam, 'existiert bereits')
    
assert os.path.isfile(bzfile_filename)

dewiktionary-latest-pages-articles-multistream.xml.bz2 existiert bereits
time: 33 ms


In [7]:
# Vorbereiten
bz          = bz2file.BZ2File(bzfile_filename)  # XML-Quelldatei
result_list = []                                # nimmt die geparsten Einträge auf
config      = {"include_wikitext": True}

time: 39 ms


## Parsen

In [8]:
# Parsen

title_last     = ''
page_part_last = 0
page_base      = 0

for num, record in enumerate(  wiktionary_de_parser.Parser(bz, config=config)  ):
    
    # num:    ID des Eintrags
    # record: Der Eintrag selbst
    
    # title
    if 'title' in record:
        title = record['title']
    else:
        title = ''
        print('no title')
        
    # page_id und page_part    
    if title == title_last:
        page_id   = page_base
        page_part = page_part_last + 1
    else:
        page_id   = num   # fortlaufende Nummer
        page_base = num
        page_part = 0
        
    
    # Am Anfang was auslassen?
    if page_id < min_id:
        continue        
       
    # Vorzeitig abbrechen?    
    if page_id > max_id:
        break
    
    # Fremdsprachiges überspringen
    if 'lang_code' in record   and   record['lang_code'] != 'de': 
        continue
    if 'lang' in record   and   record['lang'] != 'Deutsch': 
        continue        
    
    # record ergänzen
    record['page_id']     = page_id
    record['page_part']   = page_part    
    
    # last
    title_last     = title
    page_part_last = page_part    
    
    # Statusanzeige
    if page_id % 1000 == 0:    
        clear_output()
        print( page_id, title )
    
    # fürs Debugging aufbewahren
    #if num in catch_debug:
    #    debug_list.append(record)
    
    # Ausgabe
    result_list.append(record)
   
    

1118000 Sebaldinen
time: 12min 42s


## In Dataframe wandeln

In [9]:
# In DataFrame wandeln
wiktionary = pak.dataframe(result_list,verbose=False)

time: 2.71 s


In [10]:
# Daten ergänzen
wiktionary['section_len'] = wiktionary.wikitext.str.len() # Volumen der Seite
wiktionary['error'] = ''

time: 485 ms


In [11]:
# Spalten sortieren
wiktionary = pak.move_cols( wiktionary, ['page_id','page_part','title','lemma'])
wiktionary = pak.move_cols( wiktionary, ['section_len','flexion','ipa','syllables','lang','lang_code','wikitext','error'], to=-1)

time: 1.14 s


In [12]:
grid(wiktionary,pak.sample)

12 rows out of 842466


time: 10.4 s


In [13]:
# suche_debug 
mask = wiktionary.lemma.isin(suche_debug)
grid(wiktionary[mask].sort_values(['page_id',]))

19 rows


time: 226 ms


## Reinigen

In [14]:
# bj_nlp.preprocess ausprobieren
bj_nlp.preprocess('Hälló?)()')

'Hällo?)()'

time: 40.4 ms


In [15]:
wiktionary.title = wiktionary.title.apply( lambda x: bj_nlp.preprocess(x)         ) #.copy()
wiktionary.lemma = wiktionary.lemma.apply( lambda x: bj_nlp.preprocess(x)         ) #.copy()

time: 15.5 s


## section_id2 und section_id

In [16]:
# section_id2: Das gleiche wie section_id, aber verbose
wiktionary['section_id2'] = wiktionary.title.str.replace(' ','_')

mask = (wiktionary.page_part > 0)
wiktionary.loc[mask,'section_id2'] = wiktionary[mask].section_id2 + '_' + wiktionary[mask].page_part.map(str)

time: 894 ms


In [17]:
# Duplikate anzeigen (werden gleich gelöscht)
# Dups entstehen durch Akzente, die entfernt werden
dupmask = wiktionary.duplicated(subset=['section_id2'], keep=False)
result = wiktionary[dupmask].sort_values(['title','section_len'])
result = pak.drop_cols(result,['lang', 'lang_code','ipa','wikitext'])
mask = result.title.str.len() > 2
grid( result, mask, color='green' )

1370 rows out of 1433


time: 267 ms


In [18]:
# Nach dem Reinigen sind die title nicht mehr eindeutig gewesen. 
# Wir verwenden nur die jeweils längste Seite.
# Danach ist section_id2 eindeutig.
len_pre = wiktionary.shape[0]
wiktionary = pak.rank(  wiktionary, cols_group=['section_id2'], col_score='section_len')
assert wiktionary.section_id2.is_unique
print( len_pre - wiktionary.shape[0],'Datensätze gelöscht')

rank: 721 rows less, now 841745 rows
721 Datensätze gelöscht
time: 3.39 s


In [19]:
# section_id 
wiktionary['section_id'] = wiktionary.page_id * 100 + wiktionary.page_part
assert wiktionary.section_id.is_unique

time: 79 ms


In [20]:
# Duplikate anzeigen
dupmask = wiktionary.duplicated(subset=['section_id'], keep=False)
grid( wiktionary[dupmask].sort_values('title'), color='green'   )

No rows
time: 85.8 ms


In [21]:
# Zeilen wieder sortieren
wiktionary = wiktionary.sort_values(['section_id'])

time: 713 ms


In [22]:
# Spalten in Ordnung bringen
wiktionary = pak.move_cols(wiktionary, 'section_id')
wiktionary = pak.move_cols(wiktionary, 'section_id2', to='page_part')

time: 601 ms


In [23]:
grid(wiktionary.head(100))

100 rows


time: 98.9 ms


## Erste Korrekturen

In [24]:
mask1 = wiktionary.title == wiktionary.lemma
mask2A = wiktionary.pos.str.contains('Deklinierte Form')
mask2B = wiktionary.pos.str.contains('Konjugierte Form')
mask = mask1  &  (mask2A | mask2B)
wiktionary[mask]

,section_id,page_id,page_part,section_id2,title,lemma,inflected,pos,rhymes,section_len,flexion,ipa,syllables,lang,lang_code,wikitext,error


time: 2.78 s


## Fehler finden

In [25]:
# Fehler: title zu kurz
mask = (wiktionary.title.str.len() < 2)  
check_mask(wiktionary, mask, 0, 800)
wiktionary.loc[mask,'error'] = 'title zu kurz'
#wiktionary = pak.drop_rows(wiktionary,mask)

check_mask:  206 rows
time: 251 ms


In [26]:
# Fehler: page_part > 9
mask = (wiktionary.page_part > 9)   &   (wiktionary.error == '')
check_mask( wiktionary, mask, 0, 10 )
wiktionary.loc[mask,'error'] = 'page_part > 9'

check_mask:  0 rows
time: 75.5 ms


In [27]:
# Fehler finden: Uneindeutige Zuordnung
assert pak.same_but_different(wiktionary, ['page_id'], 'title').shape[0] == 0
assert pak.same_but_different(wiktionary, ['title'],   'page_id').shape[0] < 10  # es gibt ein paar Ausnahmen. Das kommt durch die Bereinigung. Ist aber egal.

time: 1.64 s


In [28]:
# Fehler finden: Uneindeutige Zuordnung
assert pak.same_but_different(wiktionary, ['section_id'],  'section_id2').shape[0] == 0
assert pak.same_but_different(wiktionary, ['section_id2'], 'section_id').shape[0] == 0

time: 1.66 s


In [29]:
# lang_code ist nan
mask = wiktionary.lang_code.isna()   &   (wiktionary.error == '')
check_mask(wiktionary, mask, 0, 9000)
wiktionary.loc[mask,'error'] = 'lang_code ist nan'
#grid(wiktionary, mask)

check_mask:  10 rows
time: 103 ms


In [30]:
# lang_code != de
mask = (wiktionary.lang_code != 'de')   &   (wiktionary.error == '')
check_mask(wiktionary, mask, 0, 10)
wiktionary.loc[mask,'error'] = 'lang_code != de'

check_mask:  0 rows
time: 122 ms


In [31]:
# lang ist nan
mask = wiktionary.lang.isna()   &   (wiktionary.error == '')
check_mask(wiktionary, mask, 0, 200)
wiktionary.loc[mask,'error'] = 'lang ist nan'
#grid(wiktionary, mask)

check_mask:  0 rows
time: 170 ms


In [32]:
# lang != Deutsch
mask = (wiktionary.lang != 'Deutsch')   &  (wiktionary.error == '')
check_mask(wiktionary, mask, 0, 10000)
wiktionary.loc[mask,'error'] = 'lang != Deutsch'

check_mask:  0 rows
time: 192 ms


In [33]:
# title zu viele Wörter
mask = (wiktionary.title.str.split().str.len() > 2)   &  (wiktionary.error == '')
check_mask(wiktionary, mask, 0, 4000)
wiktionary.loc[mask,'error'] = 'title zu viele Wörter'

check_mask:  3910 rows
time: 611 ms


In [34]:
# lemma zu kurz
mask = (wiktionary.lemma.str.len() < 2)   &  (wiktionary.error == '')
check_mask(wiktionary, mask, 0, 100)
wiktionary.loc[mask,'error'] = 'lemma zu kurz'
#grid(wiktionary,mask)

check_mask:  26 rows
time: 294 ms


In [35]:
# pos ist nan
mask = wiktionary.pos.isna()   &   (wiktionary.error == '')
check_mask(wiktionary, mask, 0, 100)
wiktionary.loc[mask,'error'] = 'pos ist nan'
#grid(wiktionary, mask)

check_mask:  0 rows
time: 167 ms


In [36]:
# pos zu kurz
#mask = (wiktionary.pos.str.len() < 2)   &  wiktionary.error.isnull()
#check_mask(wiktionary, mask, 0, 10)
#wiktionary.loc[mask,'error'] = 'pos zu kurz'

time: 31.2 ms


## Trash

In [37]:
# Fehler aussortieren
mask = wiktionary.error != ''
wiktionary, wiktionary_trash = pak.move_rows( wiktionary, mask )       

Move 4152 rows from 841745
time: 576 ms


In [38]:
pak.analyse_freqs(wiktionary_trash.error)

,item,item_count,item_percent,graph
0,title zu viele Wörter,3910,94.2,###############################################
1,title zu kurz,206,5.0,##
2,lemma zu kurz,26,0.6,
3,lang_code ist nan,10,0.2,


time: 43.2 ms


In [39]:
# Bestimmten Fehler anzeigen
# mask = wiktionary_trash.error == 'lang_code ist nan'
mask = wiktionary_trash.error == 'lemma zu kurz'
grid(wiktionary_trash, mask)

26 rows out of 4152


time: 158 ms


## nnan

In [40]:
pak.nnan(wiktionary)

rhymes       527175
flexion      713384
ipa           14004
syllables      4949
dtype: int64

time: 1.58 s


## Spalten löschen

In [41]:
# lang, lang_code, ipa, syllables
assert wiktionary.lang.nunique() == 1
assert wiktionary.lang_code.nunique() == 1
#wiktionary = pak.drop_cols(wiktionary,      ['lang','lang_code','inflected'])

time: 238 ms


## Speichern

In [42]:
# Keine Leerzeichen in der ID
mask = wiktionary.section_id2.str.contains(' ')
check_mask(wiktionary,mask,0)

check_mask:  0 rows
time: 295 ms


In [43]:
if wiktionary_filename:
    pak.dump_pickle(wiktionary, wiktionary_filename) 

time: 31.9 s


In [44]:
if wiktionary_trash_filename   and   wiktionary_trash.shape[0] > 0:
    pak.dump_pickle(wiktionary_trash, wiktionary_trash_filename)    

time: 258 ms


In [50]:
# Ausgabeverzeichnis touch
import os
os.utime(verzeichnis_save)

time: 49 ms
Executing shutdown due to inactivity...


2023-02-16 21:21:21,136 - INFO     - Executing shutdown due to inactivity...


Executing shutdown...


2023-02-16 21:21:21,154 - INFO     - Executing shutdown...


Exception on /shutdown [GET]
Traceback (most recent call last):
  File "/home/me/mambaforge/envs/python_06/lib/python3.9/site-packages/flask/app.py", line 2525, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/me/mambaforge/envs/python_06/lib/python3.9/site-packages/flask/app.py", line 1822, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/me/mambaforge/envs/python_06/lib/python3.9/site-packages/flask/app.py", line 1820, in full_dispatch_request
    rv = self.dispatch_request()
  File "/home/me/mambaforge/envs/python_06/lib/python3.9/site-packages/flask/app.py", line 1796, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
  File "/home/me/mambaforge/envs/python_06/lib/python3.9/site-packages/dtale/app.py", line 435, in shutdown
    shutdown_server()
  File "/home/me/mambaforge/envs/python_06/lib/python3.9/site-packages/dtale/app.py", line 421, in shutdown_server
    raise RuntimeError("Not 

2023-02-16 21:21:21,156 - ERROR    - Exception on /shutdown [GET]
Traceback (most recent call last):
  File "/home/me/mambaforge/envs/python_06/lib/python3.9/site-packages/flask/app.py", line 2525, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/me/mambaforge/envs/python_06/lib/python3.9/site-packages/flask/app.py", line 1822, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/me/mambaforge/envs/python_06/lib/python3.9/site-packages/flask/app.py", line 1820, in full_dispatch_request
    rv = self.dispatch_request()
  File "/home/me/mambaforge/envs/python_06/lib/python3.9/site-packages/flask/app.py", line 1796, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
  File "/home/me/mambaforge/envs/python_06/lib/python3.9/site-packages/dtale/app.py", line 435, in shutdown
    shutdown_server()
  File "/home/me/mambaforge/envs/python_06/lib/python3.9/site-packages/dtale/app.py", line 421, in shutdow

In [45]:
# Save in Google Drive
if environment['in_colab']:
    from google.colab import drive
    drive.mount('/content/drive') 
    !cd /content
    !cp -av {verzeichnis_save} /content/drive/MyDrive 

time: 33.5 ms


## Kontrolle

In [46]:
# suche_debug
mask = wiktionary.title.isin(suche_debug)
grid(wiktionary, mask)

1 rows out of 837593


time: 153 ms


In [47]:
# histo section_len
a = (wiktionary.section_len / 10000).round().astype('int') * 1000
pak.analyse_freqs(a, sort_count=False)

,item,item_count,item_percent,graph
0,0,830604,99.2,#################################################
1,1000,6701,0.8,
2,2000,223,0.0,
3,3000,37,0.0,
4,4000,12,0.0,
5,5000,7,0.0,
6,6000,3,0.0,
7,7000,3,0.0,
8,8000,2,0.0,
9,9000,1,0.0,


time: 221 ms


In [48]:
# page_part
pak.analyse_freqs(wiktionary.page_part, sort_count=False)

,item,item_count,item_percent,graph
0,0,802084,95.8,###############################################
1,1,33817,4.0,##
2,2,1452,0.2,
3,3,205,0.0,
4,4,29,0.0,
5,5,5,0.0,
6,6,1,0.0,


time: 56.1 ms


In [49]:
raise Stop

Stop Time:  20:21:20
Elapsed:    14 mins, 9 secs


time: 49.6 ms
